# Debug implementations for slab waveguide class

In [ ]:
import numpy as np
from numpy import exp
from fiberamp.fiber.microstruct.bragg import plotlogf, plotlogf_real

In [ ]:
n0 = 1.3
n1 = 1.5
rho = 1
wl = 1.8
k0 = 2*np.pi/wl

In [ ]:
def guided_det(beta, rho=rho, n0=n0, n1=n1):
    beta = np.array(beta, dtype=complex)
    z0 = 1j*np.sqrt(-k0**2 * n0**2 + beta**2, dtype=complex)
    z1 = np.sqrt(k0**2 * n1**2 - beta**2, dtype=complex)
    A = (z0*n1**2 + z1*n0**2)**2
    B = (z0*n1**2 - z1*n0**2)**2
    
    return (np.exp(2j*rho*(z0-z1))*A - np.exp(2j*rho*(z1+z0))*B) / (4*z0*z1*(n0*n1)**2)

def leaky_det(beta, rho=rho, n0=n0, n1=n1):
    beta = np.array(beta, dtype=complex)
    z0 = 1j*np.sqrt(-k0**2 * n0**2 + beta**2, dtype=complex)
    z1 = np.sqrt(k0**2 * n1**2 - beta**2, dtype=complex)
    A = (z0*n1**2 + z1*n0**2)**2
    B = (z0*n1**2 - z1*n0**2)**2
    
    return (np.exp(2j*rho*(z1-z0))*A - np.exp(-2j*rho*(z1+z0))*B) / (4*z0*z1*(n0*n1)**2)


def guided_det_TE(beta, rho=rho, n0=n0, n1=n1):
    beta = np.array(beta, dtype=complex)
    z0 = 1j*np.sqrt(-k0**2 * n0**2 + beta**2, dtype=complex)
    z1 = np.sqrt(k0**2 * n1**2 - beta**2, dtype=complex)
    A = (z0 + z1)**2
    B = (z0 - z1)**2
    
    return (np.exp(2j*rho*(z0-z1))*A - np.exp(2j*rho*(z1+z0))*B) / (4*z0*z1)

def leaky_det_TE(beta, rho=rho, n0=n0, n1=n1):
    beta = np.array(beta, dtype=complex)
    z0 = 1j*np.sqrt(-k0**2 * n0**2 + beta**2, dtype=complex)
    z1 = np.sqrt(k0**2 * n1**2 - beta**2, dtype=complex)
    A = (z0 + z1)**2
    B = (z0 - z1)**2
    
    return (np.exp(2j*rho*(z1-z0))*A - np.exp(-2j*rho*(z1+z0))*B) / (4*z0*z1)

In [ ]:
plotlogf(guided_det, .95*k0*n0, 1.05*k0*n1, -.3,.3, iref=100, rref=100,levels=60)

In [ ]:
plotlogf(leaky_det, .3*k0*n0, 1.1*k0*n0, -1.5,1.5, iref=100, rref=100,levels=60)

In [ ]:

def TransferMatrix_TM(beta, rho, n_left, n_right, k0=k0, 
                      Ztype_left='standard', Ztype_right='standard'):
    
    beta = np.array(beta, dtype=complex)
    
    if Ztype_left == 'standard':
        Z_left = np.sqrt(k0**2 * n_left**2 - beta**2)
    elif Ztype_left == 'imag':
        Z_left = 1j * np.sqrt(beta**2 - k0**2 * n_left**2)
    else:
        raise ValueError('Ztype must be standard or imag.')

    if Ztype_right == 'standard':
        Z_right = np.sqrt(k0**2 * n_right**2 - beta**2)
    elif Ztype_right == 'imag':
        Z_right = 1j * np.sqrt(beta**2 - k0**2 * n_right**2)
    else:
        raise ValueError('Ztype must be standard or imag.')

    Exp_minus = 1j * (Z_right - Z_left) * rho
    Exp_plus = 1j * (Z_right + Z_left) * rho

    A_minus = Z_right * n_left**2 - Z_left * n_right**2
    A_plus = Z_right * n_left**2 + Z_left * n_right**2

    M = np.array([[exp(-Exp_minus) * A_plus, exp(-Exp_plus) * A_minus],
                [exp(Exp_plus) * A_minus, exp(Exp_minus) * A_plus]])
    return M * 1 / (2 * Z_right * n_left**2)


def TransferMatrix_TE(beta, rho, n_left, n_right, k0=k0,
                      Ztype_left='standard', Ztype_right='standard'):
    
    beta = np.array(beta, dtype=complex)
    
    if Ztype_left == 'standard':
        Z_left = np.sqrt(k0**2 * n_left**2 - beta**2)
    elif Ztype_left == 'imag':
        Z_left = 1j * np.sqrt(beta**2 - k0**2 * n_left**2)
    else:
        raise ValueError('Ztype must be standard or imag.')

    if Ztype_right == 'standard':
        Z_right = np.sqrt(k0**2 * n_right**2 - beta**2)
    elif Ztype_right == 'imag':
        Z_right = 1j * np.sqrt(beta**2 - k0**2 * n_right**2)
    else:
        raise ValueError('Ztype must be standard or imag.')

    Exp_minus = 1j * (Z_right - Z_left) * rho
    Exp_plus = 1j * (Z_right + Z_left) * rho

    A_minus = Z_right - Z_left
    A_plus = Z_right + Z_left

    M = np.array([[exp(-Exp_minus) * A_plus, exp(-Exp_plus) * A_minus],
                [exp(Exp_plus) * A_minus, exp(Exp_minus) * A_plus]])
    return M * 1 / (2 * Z_right)


In [ ]:
def det2(beta, rho=rho, n0=n0, n1=n1):
        beta = np.array(beta, dtype=complex)
        M = TransferMatrix_TM(beta, rho, n1, n0, Ztype_right='imag') @ TransferMatrix_TM(beta, -rho, n0, n1, Ztype_left='imag')
        return M[1,1]
    
def det_TE(beta, rho=rho, n0=n0, n1=n1):
        beta = np.array(beta, dtype=complex)
        M = TransferMatrix_TE(beta, rho, n1, n0, Ztype_right='imag') @ TransferMatrix_TE(beta, -rho, n0, n1, Ztype_left='imag')
        return M[1,1]

In [ ]:
k0*n0, k0*n1

In [ ]:
plotlogf(det2, k0*n0, k0*n1, -.2, .2, iref=100, rref=100,levels=100, loop=True)

In [ ]:
plotlogf_real(det2, 4.6, 5.2, loop=True, log_off=True, abs_off=True)

In [ ]:
plotlogf_real(guided_det, 4.6, 5.2, log_off=True, abs_off=True)

In [ ]:
from scipy.optimize import bisect, newton

In [ ]:
beta1_TM = newton(det2, 4.7)
beta1_TM

In [ ]:
beta2_TM = newton(det2, 5)
beta2_TM

### Retrieve Fields

Time to see if we're doing this right.  For rightmost region we need A0=0, for left most need Bn=0.  Pick B0=1

In [ ]:
B0 = 1
M0_TM1 = np.array([0, B0])

In [ ]:
beta = beta1_TM
M1_TM1 = TransferMatrix_TM(beta, -rho, n0, n1, Ztype_left='imag') @ M0_TM1
M2_TM1 = TransferMatrix_TM(beta, rho, n1, n0, Ztype_right='imag') @ M1_TM1

In [ ]:
TransferMatrix_TM(beta, -rho, n0, n1, Ztype_left='imag')

In [ ]:
beta

In [ ]:
rho, n0, n1

In [ ]:
T = TransferMatrix_TM(beta, -rho, n0, n1, Ztype_left='imag')
T

In [ ]:
M0_TM1, M1_TM1, M2_TM1

In [ ]:
xL = np.linspace(-3,-1, 1000)
xC = np.linspace(-1,1, 1000)
xR = np.linspace(1,3, 1000)

In [ ]:
def R(xs, beta, M, n, k0=k0, Ztype='standard'):
    
    A, B = M[:]
    
    if Ztype == 'standard':
        Z = np.sqrt(k0**2 * n**2 - beta**2)
    elif Ztype == 'imag':
        Z = 1j * np.sqrt(beta**2 - k0**2 * n**2)
    else:
        raise ValueError('Ztype must be standard or imag.')  
    
        
    return A * np.exp(1j*Z*xs) + B * np.exp(-1j*Z*xs)

In [ ]:
yL_TM1 = R(xL, beta, M0_TM1, n0, Ztype='imag')
yC_TM1 = R(xC, beta, M1_TM1, n1)
yR_TM1 = R(xR, beta, M2_TM1, n0, Ztype='imag')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(xL, yL_TM1.real);
plt.plot(xC, yC_TM1.real);
plt.plot(xR, yR_TM1.real);

Other beta

In [ ]:
B0 = 1
M0_TM2 = np.array([0,B0])

In [ ]:
beta = beta2_TM
M1_TM2 = TransferMatrix_TM(beta, -rho, n0, n1, Ztype_left='imag') @ M0_TM2
M2_TM2 = TransferMatrix_TM(beta, rho, n1, n0, Ztype_right='imag') @ M1_TM2

In [ ]:
M0_TM2, M1_TM2, M2_TM2

In [ ]:
xL = np.linspace(-3,-1, 3000)
xC = np.linspace(-1,1, 3000)
xR = np.linspace(1,3, 3000)

In [ ]:
def R(xs, beta, M, n, k0=k0, Ztype='standard'):
    
    A, B = M[:]
    
    if Ztype == 'standard':
        Z = np.sqrt(k0**2 * n**2 - beta**2, dtype=complex)
    elif Ztype == 'imag':
        Z = 1j * np.sqrt(beta**2 - k0**2 * n**2, dtype=complex)
    else:
        raise ValueError('Ztype must be standard or imag.')  
    
        
    return A * np.exp(1j*Z*xs) + B * np.exp(-1j*Z*xs)

In [ ]:
yL_TM2 = R(xL, beta, M0_TM2, n0, Ztype='imag')
yC_TM2 = R(xC, beta, M1_TM2, n1)
yR_TM2 = R(xR, beta, M2_TM2, n0, Ztype='imag')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(xL, yL_TM2.real);
plt.plot(xC, yC_TM2.real);
plt.plot(xR, yR_TM2.real);

Leaky modes?  Need B0=0 now.

In [ ]:
plotlogf(leaky_det, 4,5, -.5,.5, iref=100, rref=100,levels=60)

In [ ]:
beta3_TM = newton(leaky_det, np.array([4.6+.2j]))

In [ ]:
A0 = 1
M0_TM3 = np.array([A0, 0])

In [ ]:

beta = beta3_TM[0]
M1_TM3 = TransferMatrix_TM(beta, -rho, n0, n1, Ztype_left='imag') @ M0_TM3
M2_TM3 = TransferMatrix_TM(beta, rho, n1, n0, Ztype_right='imag') @ M1_TM3

In [ ]:
xL = np.linspace(-3,-1, 1000)
xC = np.linspace(-1,1, 1000)
xR = np.linspace(1,3, 1000)

In [ ]:
def R(xs, beta, M, n, k0=k0, Ztype='standard'):
    
    A, B = M[:]
    
    if Ztype == 'standard':
        Z = np.sqrt(k0**2 * n**2 - beta**2, dtype=complex)
    elif Ztype == 'imag':
        Z = 1j * np.sqrt(beta**2 - k0**2 * n**2, dtype=complex)
    else:
        raise ValueError('Ztype must be standard or imag.')  
    
        
    return A * np.exp(1j*Z*xs) + B * np.exp(-1j*Z*xs)

In [ ]:
yL_TM3 = R(xL, beta, M0_TM3, n0, Ztype='imag')
yC_TM3 = R(xC, beta, M1_TM3, n1)
yR_TM3 = R(xR, beta, M2_TM3, n0, Ztype='imag')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(xL, yL_TM3.real);
plt.plot(xC, yC_TM3.real);
plt.plot(xR, yR_TM3.real);

In [ ]:
plt.plot(xL, yL_TM3.imag);
plt.plot(xC, yC_TM3.imag);
plt.plot(xR, yR_TM3.imag);

# Do it for TE case now

In [ ]:
plotlogf(guided_det_TE, .95*k0*n0, 1.05*k0*n1, -.3,.3, iref=100, rref=100,levels=60)

In [ ]:
plotlogf(leaky_det_TE, .3*k0*n0, 1.1*k0*n0, -1.5,1.5, iref=100, rref=100,levels=60)

In [ ]:
plotlogf(det_TE, k0*n0, k0*n1, -.2, .2, iref=100, rref=100,levels=100, loop=True)

In [ ]:
plotlogf_real(det_TE, 4.6, 5.2, loop=True, log_off=True, abs_off=True)

In [ ]:
plotlogf_real(guided_det_TE, 4.6, 5.2, log_off=True, abs_off=True)

In [ ]:
from scipy.optimize import bisect, newton

In [ ]:
beta1_TE = newton(lambda x: guided_det_TE(x).real, 4.7)
beta1_TE, beta1_TM

In [ ]:
beta2_TE = newton(lambda x: guided_det_TE(x).real, 5)
beta2_TE, beta2_TM

### Retrieve Fields

Time to see if we're doing this right.  For rightmost region we need A0=0, for left most need Bn=0.  Pick B0=1

In [ ]:
B0 = 1
M0_TE1 = np.array([0,B0])

In [ ]:
beta = beta1_TE
M1_TE1 = TransferMatrix_TE(beta, -rho, n0, n1, Ztype_left='imag') @ M0_TE1
M2_TE1 = TransferMatrix_TE(beta, rho, n1, n0, Ztype_right='imag') @ M1_TE1

In [ ]:
xL = np.linspace(-3,-1, 1000)
xC = np.linspace(-1,1, 1000)
xR = np.linspace(1,3, 1000)

In [ ]:
def R(xs, beta, M, n, k0=k0, Ztype='standard'):
    
    A, B = M[:]
    
    if Ztype == 'standard':
        Z = np.sqrt(k0**2 * n**2 - beta**2)
    elif Ztype == 'imag':
        Z = 1j * np.sqrt(beta**2 - k0**2 * n**2)
    else:
        raise ValueError('Ztype must be standard or imag.')  
    
        
    return A * np.exp(1j*Z*xs) + B * np.exp(-1j*Z*xs)

In [ ]:
yL_TE1 = R(xL, beta, M0_TE1, n0, Ztype='imag')
yC_TE1 = R(xC, beta, M1_TE1, n1)
yR_TE1 = R(xR, beta, M2_TE1, n0, Ztype='imag')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(xL, yL_TM1.real, color='g');
plt.plot(xC, yC_TM1.real, color='g');
plt.plot(xR, yR_TM1.real, color='g', label='TM');
plt.plot(xL, yL_TE1.real, color='r', label='TE');
plt.plot(xC, yC_TE1.real, color='r');
plt.plot(xR, yR_TE1.real, color='r');
plt.legend()

Other beta

In [ ]:
B0 = 1
M0_TE2 = np.array([0,B0])

In [ ]:
beta = beta2_TE
M1_TE2 = TransferMatrix_TE(beta, -rho, n0, n1, Ztype_left='imag') @ M0_TE2
M2_TE2 = TransferMatrix_TE(beta, rho, n1, n0, Ztype_right='imag') @ M1_TE2

In [ ]:
xL = np.linspace(-3,-1, 3000)
xC = np.linspace(-1,1, 3000)
xR = np.linspace(1,3, 3000)

In [ ]:
def R(xs, beta, M, n, k0=k0, Ztype='standard'):
    
    A, B = M[:]
    
    if Ztype == 'standard':
        Z = np.sqrt(k0**2 * n**2 - beta**2, dtype=complex)
    elif Ztype == 'imag':
        Z = 1j * np.sqrt(beta**2 - k0**2 * n**2, dtype=complex)
    else:
        raise ValueError('Ztype must be standard or imag.')  
    
        
    return A * np.exp(1j*Z*xs) + B * np.exp(-1j*Z*xs)

In [ ]:
yL_TE2 = R(xL, beta, M0_TE2, n0, Ztype='imag')
yC_TE2 = R(xC, beta, M1_TE2, n1)
yR_TE2 = R(xR, beta, M2_TE2, n0, Ztype='imag')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(xL, yL_TM2.real, color='g');
plt.plot(xC, yC_TM2.real, color='g');
plt.plot(xR, yR_TM2.real, color='g', label='TM');
plt.plot(xL, yL_TE2.real, color='r', label='TE');
plt.plot(xC, yC_TE2.real, color='r');
plt.plot(xR, yR_TE2.real, color='r');
plt.legend()

Leaky modes?  Need B0=0 now.

In [ ]:
plotlogf(leaky_det_TE, 4,5, -.5,.5, iref=100, rref=100,levels=60)

In [ ]:
beta3_TE = newton(leaky_det_TE, np.array([4.6+.2j]))
beta3_TE, beta3_TM

In [ ]:
A0 = 1
M0_TE3 = np.array([A0, 0])

In [ ]:

beta = beta3_TE[0]
M1_TE3 = TransferMatrix_TE(beta, -rho, n0, n1, Ztype_left='imag') @ M0_TE3
M2_TE3 = TransferMatrix_TE(beta, rho, n1, n0, Ztype_right='imag') @ M1_TE3

In [ ]:
xL = np.linspace(-3,-1, 1000)
xC = np.linspace(-1,1, 1000)
xR = np.linspace(1,3, 1000)

In [ ]:
def R(xs, beta, M, n, k0=k0, Ztype='standard'):
    
    A, B = M[:]
    
    if Ztype == 'standard':
        Z = np.sqrt(k0**2 * n**2 - beta**2, dtype=complex)
    elif Ztype == 'imag':
        Z = 1j * np.sqrt(beta**2 - k0**2 * n**2, dtype=complex)
    else:
        raise ValueError('Ztype must be standard or imag.')  
    
        
    return A * np.exp(1j*Z*xs) + B * np.exp(-1j*Z*xs)

In [ ]:
yL_TE3 = R(xL, beta, M0_TE3, n0, Ztype='imag')
yC_TE3 = R(xC, beta, M1_TE3, n1)
yR_TE3 = R(xR, beta, M2_TE3, n0, Ztype='imag')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(xL, yL_TM3.real, color='g');
plt.plot(xC, yC_TM3.real, color='g');
plt.plot(xR, yR_TM3.real, color='g', label='TM real');
plt.plot(xL, yL_TE3.real, color='r', label='TE real');
plt.plot(xC, yC_TE3.real, color='r');
plt.plot(xR, yR_TE3.real, color='r');
plt.legend();

In [ ]:
plt.plot(xL, yL_TM3.imag, color='g');
plt.plot(xC, yC_TM3.imag, color='g');
plt.plot(xR, yR_TM3.imag, color='g', label='TM imag');
plt.plot(xL, yL_TE3.imag, color='r', label='TE imag');
plt.plot(xC, yC_TE3.imag, color='r');
plt.plot(xR, yR_TE3.imag, color='r');
plt.legend();